In [107]:
import pandas as pd

datapath = "~/Documents/datasets/ford_sentence/"

In [108]:
data = pd.read_csv(datapath + "train_data.csv", skip_blank_lines=True)
data = data[data.New_Sentence.notna()]

num_sentences = len(data.iloc[:])
print(num_sentences)

59002


### Tracking number of occurences for each word

In [109]:
vocab_all = {}

for i, sentence in enumerate(data.New_Sentence):
    for word in sentence.split():
        word = word.lower()
        if word in vocab_all:
            vocab_all[word] += 1
        else:
            vocab_all[word] = 1


### Omitting words that occure less than 5 times

In [110]:
vocab = {}

for key, val in vocab_all.items():
    if val >= 5: vocab[key] = 0

### Computing probability of occurence for every word

In [111]:
for sentence in data.New_Sentence:
    added = set()
    for word in sentence.split():
        word = word.lower()
        if word in vocab.keys() and word not in added:
            vocab[word] += 1        
            added.add(word)

for key in vocab.keys():
    vocab[key] /= num_sentences